In [ ]:
f = open('/content/drive/MyDrive/SudoCode/Week 1 - Text Preprocessing/news_dataset.json',encoding = 'utf-8')

In [ ]:
raw_data = f.read()

In [ ]:
raw_data[:200]

'[\n\t{\n\t\t"id" : 218270,\n\t\t"author" : "",\n\t\t"content" : "Chiều 31/7, Công an tỉnh Thừa Thiên - Huế đã có thông tin ban đầu về vụ nổ súng,cướp tiệm vàng tại chợ Đông Ba nằm trên đường Trần Hưng Đạo (TP Hu'

In [ ]:
working_data = raw_data[:3166]
working_data

'[\n\t{\n\t\t"id" : 218270,\n\t\t"author" : "",\n\t\t"content" : "Chiều 31/7, Công an tỉnh Thừa Thiên - Huế đã có thông tin ban đầu về vụ nổ súng,cướp tiệm vàng tại chợ Đông Ba nằm trên đường Trần Hưng Đạo (TP Huế, tỉnh Thừa Thiên - Huế). Thông Sài Gòn Giải Phóng, khoảng 12h30\' ngày 31/7, một đối tượng sử dụng súng AK bất ngờ xông vào tiệm vàng Hoàng Đức và Thái Lợi (phía trước chợ Đông Ba) rồi nổ súng chỉ thiên liên tiếp uy hiếp chủ tiệm để cướp vàng. Sau đó, đối tượng mang số vàng vừa cướp được vứt ra vỉa hè rồi đi bộ đến khu vực cầu Gia Hội, cách khu vực gây án khoảng 300m. Giám đốc Công an tỉnh Thừa Thiên – Huế lập tức trực tiếp chỉ đạo các lực lượng chức năng gồm Công an tỉnh và Công an TP Huế nhanh chóng có mặt tại hiện trường triển khai đồng bộ các biện pháp nghiệp vụ, khoanh vùng và ngăn không để người dân đi vào hiện trường. Hàng trăm tiểu thương trong chợ Đông Ba và người dân gần cầu Gia Hội được yêu cầu di chuyển khỏi hiện trường, đóng cửa nhà đề phòng đạn lạc. Tuy nhiên, t

# Approach 1: Preprocessing with specialized tokenization

In [ ]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
import nltk, re
from nltk import word_tokenize

**Objective**: Do thấy data ở dạng raw và có cấu trúc gồm cái field như id, author, content nên em có phương hướng xử lý chung như sau

Overall:
  1. Xử lý những whitespace character như \n\t, ... etc
  2. Tokenize những thông tin đặc đi theo cụm dễ hiểu nghĩa như `url`, thời gian timestamp không tính ngày tháng nghị quyết, email, đơn vị (300m, 12s, ...)
  3. Xử lý `crawled_at` thành tokens theo kiểu ví dụ:
     `"2022-08-01 09:08:13.106296" -> ['2022-08-01', '09:08:13.106296']`

In [ ]:
clean_re = re.sub(r'[\t\n\[\]\{\}\'\"]*', '', working_data).lower()
clean_re

'id : 218270,author : ,content : chiều 31/7, công an tỉnh thừa thiên - huế đã có thông tin ban đầu về vụ nổ súng,cướp tiệm vàng tại chợ đông ba nằm trên đường trần hưng đạo (tp huế, tỉnh thừa thiên - huế). thông sài gòn giải phóng, khoảng 12h30 ngày 31/7, một đối tượng sử dụng súng ak bất ngờ xông vào tiệm vàng hoàng đức và thái lợi (phía trước chợ đông ba) rồi nổ súng chỉ thiên liên tiếp uy hiếp chủ tiệm để cướp vàng. sau đó, đối tượng mang số vàng vừa cướp được vứt ra vỉa hè rồi đi bộ đến khu vực cầu gia hội, cách khu vực gây án khoảng 300m. giám đốc công an tỉnh thừa thiên – huế lập tức trực tiếp chỉ đạo các lực lượng chức năng gồm công an tỉnh và công an tp huế nhanh chóng có mặt tại hiện trường triển khai đồng bộ các biện pháp nghiệp vụ, khoanh vùng và ngăn không để người dân đi vào hiện trường. hàng trăm tiểu thương trong chợ đông ba và người dân gần cầu gia hội được yêu cầu di chuyển khỏi hiện trường, đóng cửa nhà đề phòng đạn lạc. tuy nhiên, thấy vàng bị ném ra đường, nhiều ngư

In [ ]:
# Giữ lại website url, thời gian timestamp không tính ngày tháng nghị quyết, email, đơn vị (300m)
pattern = r'''(?x)     # set flag to allow verbose regexps
          (?:(?:\w+[ ]?\.[ ]?)+[\d]+)
          | (?:https?://[\w\d\.-]+(?:/[^\s]*)?|www\.[\w\d\.-]+) # Website url
          | (?:\d+[a-z]{1}\d*)                                  # Unit + thời gian như 1h30
          | (?:\d{2}:\d{2}:\d+.?\d+)                            # timestamp
          | [\w\.-]+@[\w\.-]+\.\w+                              # email
          | (?:\w+(?:[/-]\w+)+)                                 # slash/dash combos (2372/gp-stttt)
          | \w+                                                 # fallback: plain alphanumeric
          | \d+(?:[/-]\d+)+
'''
nltk.regexp_tokenize(clean_re, pattern)

['id',
 '218270',
 'author',
 'content',
 'chiều',
 '31/7',
 'công',
 'an',
 'tỉnh',
 'thừa',
 'thiên',
 'huế',
 'đã',
 'có',
 'thông',
 'tin',
 'ban',
 'đầu',
 'về',
 'vụ',
 'nổ',
 'súng',
 'cướp',
 'tiệm',
 'vàng',
 'tại',
 'chợ',
 'đông',
 'ba',
 'nằm',
 'trên',
 'đường',
 'trần',
 'hưng',
 'đạo',
 'tp',
 'huế',
 'tỉnh',
 'thừa',
 'thiên',
 'huế',
 'thông',
 'sài',
 'gòn',
 'giải',
 'phóng',
 'khoảng',
 '12h30',
 'ngày',
 '31/7',
 'một',
 'đối',
 'tượng',
 'sử',
 'dụng',
 'súng',
 'ak',
 'bất',
 'ngờ',
 'xông',
 'vào',
 'tiệm',
 'vàng',
 'hoàng',
 'đức',
 'và',
 'thái',
 'lợi',
 'phía',
 'trước',
 'chợ',
 'đông',
 'ba',
 'rồi',
 'nổ',
 'súng',
 'chỉ',
 'thiên',
 'liên',
 'tiếp',
 'uy',
 'hiếp',
 'chủ',
 'tiệm',
 'để',
 'cướp',
 'vàng',
 'sau',
 'đó',
 'đối',
 'tượng',
 'mang',
 'số',
 'vàng',
 'vừa',
 'cướp',
 'được',
 'vứt',
 'ra',
 'vỉa',
 'hè',
 'rồi',
 'đi',
 'bộ',
 'đến',
 'khu',
 'vực',
 'cầu',
 'gia',
 'hội',
 'cách',
 'khu',
 'vực',
 'gây',
 'án',
 'khoảng',
 '300m',
 'giám'

In [ ]:
clean_raw_data_nltk = nltk.wordpunct_tokenize(clean_re)
clean_raw_data_nltk

['id',
 ':',
 '218270',
 ',',
 'author',
 ':',
 ',',
 'content',
 ':',
 'chiều',
 '31',
 '/',
 '7',
 ',',
 'công',
 'an',
 'tỉnh',
 'thừa',
 'thiên',
 '-',
 'huế',
 'đã',
 'có',
 'thông',
 'tin',
 'ban',
 'đầu',
 'về',
 'vụ',
 'nổ',
 'súng',
 ',',
 'cướp',
 'tiệm',
 'vàng',
 'tại',
 'chợ',
 'đông',
 'ba',
 'nằm',
 'trên',
 'đường',
 'trần',
 'hưng',
 'đạo',
 '(',
 'tp',
 'huế',
 ',',
 'tỉnh',
 'thừa',
 'thiên',
 '-',
 'huế',
 ').',
 'thông',
 'sài',
 'gòn',
 'giải',
 'phóng',
 ',',
 'khoảng',
 '12h30',
 'ngày',
 '31',
 '/',
 '7',
 ',',
 'một',
 'đối',
 'tượng',
 'sử',
 'dụng',
 'súng',
 'ak',
 'bất',
 'ngờ',
 'xông',
 'vào',
 'tiệm',
 'vàng',
 'hoàng',
 'đức',
 'và',
 'thái',
 'lợi',
 '(',
 'phía',
 'trước',
 'chợ',
 'đông',
 'ba',
 ')',
 'rồi',
 'nổ',
 'súng',
 'chỉ',
 'thiên',
 'liên',
 'tiếp',
 'uy',
 'hiếp',
 'chủ',
 'tiệm',
 'để',
 'cướp',
 'vàng',
 '.',
 'sau',
 'đó',
 ',',
 'đối',
 'tượng',
 'mang',
 'số',
 'vàng',
 'vừa',
 'cướp',
 'được',
 'vứt',
 'ra',
 'vỉa',
 'hè',
 'rồi',


# Approach 2: Preprocessing with normalize everything - treat it the same

In [ ]:
token_nltk = nltk.word_tokenize(working_data.lower())
token_nltk

['[',
 '{',
 "''",
 'id',
 "''",
 ':',
 '218270',
 ',',
 "''",
 'author',
 "''",
 ':',
 '``',
 "''",
 ',',
 "''",
 'content',
 "''",
 ':',
 '``',
 'chiều',
 '31/7',
 ',',
 'công',
 'an',
 'tỉnh',
 'thừa',
 'thiên',
 '-',
 'huế',
 'đã',
 'có',
 'thông',
 'tin',
 'ban',
 'đầu',
 'về',
 'vụ',
 'nổ',
 'súng',
 ',',
 'cướp',
 'tiệm',
 'vàng',
 'tại',
 'chợ',
 'đông',
 'ba',
 'nằm',
 'trên',
 'đường',
 'trần',
 'hưng',
 'đạo',
 '(',
 'tp',
 'huế',
 ',',
 'tỉnh',
 'thừa',
 'thiên',
 '-',
 'huế',
 ')',
 '.',
 'thông',
 'sài',
 'gòn',
 'giải',
 'phóng',
 ',',
 'khoảng',
 '12h30',
 "'",
 'ngày',
 '31/7',
 ',',
 'một',
 'đối',
 'tượng',
 'sử',
 'dụng',
 'súng',
 'ak',
 'bất',
 'ngờ',
 'xông',
 'vào',
 'tiệm',
 'vàng',
 'hoàng',
 'đức',
 'và',
 'thái',
 'lợi',
 '(',
 'phía',
 'trước',
 'chợ',
 'đông',
 'ba',
 ')',
 'rồi',
 'nổ',
 'súng',
 'chỉ',
 'thiên',
 'liên',
 'tiếp',
 'uy',
 'hiếp',
 'chủ',
 'tiệm',
 'để',
 'cướp',
 'vàng',
 '.',
 'sau',
 'đó',
 ',',
 'đối',
 'tượng',
 'mang',
 'số',
 'vàng'

# Approach 3: Preprocessing in table - Sentence Segmentation

In [ ]:
import pandas as pd
data = pd.read_json('/content/drive/MyDrive/SudoCode/Week 1 - Text Preprocessing/news_dataset.json')

In [ ]:
data

,id,author,content,picture_count,processed,source,title,topic,url,crawled_at
0,218270,,"Chiều 31/7, Công an tỉnh Thừa Thiên - Huế đã c...",3,0,docbao.vn,"Tên cướp tiệm vàng tại Huế là đại uý công an, ...",Pháp luật,https://docbao.vn/phap-luat/ten-cuop-tiem-vang...,2022-08-01 09:09:22.817308
1,218269,(Nguồn: Sina),"Gần đây, Thứ trưởng Bộ Phát triển Kỹ thuật số,...",1,0,vtc.vn,"Bỏ qua mạng 5G, Nga tiến thẳng từ 4G lên 6G",Sống kết nối,https://vtc.vn/bo-qua-mang-5g-nga-tien-thang-t...,2022-08-01 09:09:21.181469
2,218268,Hồ Sỹ Anh,Kết quả thi tốt nghiệp THPT năm 2022 cho thấy ...,3,0,thanhnien.vn,Địa phương nào đứng đầu cả nước tổng điểm 3 mô...,Giáo dục,https://thanhnien.vn/dia-phuong-nao-dung-dau-c...,2022-08-01 09:09:15.311901
3,218267,Ngọc Ánh,Thống đốc Kentucky Andy Beshear hôm 31/7 cho h...,1,0,vnexpress,Người chết trong mưa lũ 'nghìn năm có một' ở M...,Thế giới,https://vnexpress.net/nguoi-chet-trong-mua-lu-...,2022-08-01 09:09:02.211498
4,218266,HẢI YẾN - MINH LÝ,Vụ tai nạn giao thông liên hoàn trên phố đi bộ...,12,0,soha,"Hải Phòng: Hình ảnh xe ""điên"" gây tai nạn liên...",Thời sự - Xã hội,https://soha.vn/hai-phong-hinh-anh-xe-dien-gay...,2022-08-01 09:09:01.601170
...,...,...,...,...,...,...,...,...,...,...
184534,5,Quốc Thắng,"Chiều 7/6, Khoa bị Công an quận Phú Nhuận phối...",2,1,vnexpress,Nghi can sát hại cha bị bắt,Pháp luật,https://vnexpress.net/nghi-can-sat-hai-cha-bi-...,2022-06-07 14:05:53.848085
184535,4,Thảo Mi,"Ngày 7/6, phiên xử phúc thẩm ông Tất Thành Can...",2,1,vnexpress,Ông Tất Thành Cang được đề nghị giảm án,Pháp luật,https://vnexpress.net/ong-tat-thanh-cang-duoc-...,2022-06-07 14:05:53.482540
184536,3,Thanh Tâm,"""Các khu dân cư của thành phố Severodonetsk đã...",2,1,vnexpress,Nga tuyên bố 'giải phóng' hoàn toàn khu dân cư...,Thế giới,https://vnexpress.net/nga-tuyen-bo-giai-phong-...,2022-06-07 14:05:51.797545
184537,2,Lâm Thỏa,"*Việt Nam - Malaysia: 20h thứ Tư 8/6, trên VnE...",2,1,vnexpress,HLV Malaysia dọa ngáng đường Việt Nam,Thể thao,https://vnexpress.net/hlv-malaysia-doa-ngang-d...,2022-06-07 14:05:51.433712


In [ ]:
data["sentences"] = data["content"].apply(lambda x: nltk.sent_tokenize(str(x)))

In [ ]:
data.head(3)

,id,author,content,picture_count,processed,source,title,topic,url,crawled_at,sentences
0,218270,,"Chiều 31/7, Công an tỉnh Thừa Thiên - Huế đã c...",3,0,docbao.vn,"Tên cướp tiệm vàng tại Huế là đại uý công an, ...",Pháp luật,https://docbao.vn/phap-luat/ten-cuop-tiem-vang...,2022-08-01 09:09:22.817308,"[Chiều 31/7, Công an tỉnh Thừa Thiên - Huế đã ..."
1,218269,(Nguồn: Sina),"Gần đây, Thứ trưởng Bộ Phát triển Kỹ thuật số,...",1,0,vtc.vn,"Bỏ qua mạng 5G, Nga tiến thẳng từ 4G lên 6G",Sống kết nối,https://vtc.vn/bo-qua-mang-5g-nga-tien-thang-t...,2022-08-01 09:09:21.181469,"[Gần đây, Thứ trưởng Bộ Phát triển Kỹ thuật số..."
2,218268,Hồ Sỹ Anh,Kết quả thi tốt nghiệp THPT năm 2022 cho thấy ...,3,0,thanhnien.vn,Địa phương nào đứng đầu cả nước tổng điểm 3 mô...,Giáo dục,https://thanhnien.vn/dia-phuong-nao-dung-dau-c...,2022-08-01 09:09:15.311901,[Kết quả thi tốt nghiệp THPT năm 2022 cho thấy...
